In [74]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [75]:
# todo: think about what to do with pagination
# todo: think about do we need EventCardUrgencySignal__label - going fast, sold out, almost full etc
url = 'https://www.eventbrite.com/d/portugal--ilha-da-madeira--85687345/all-events/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)
time.sleep(5)

soup = BeautifulSoup(response.text, 'html.parser')
events = soup.find_all('div', class_='discover-search-desktop-card discover-search-desktop-card--hiddeable')


In [77]:
fields = ["title", "date", "time", "location", "category"]

data = {field: [] for field in fields}

for event in events: 
    basic_info = event.find('a')
    data['title'].append(event.find('h2').text)

    p_elements = event.find_all('p')
    # remove urgency signal chip data
    filtered_elements = [p for p in p_elements if 'EventCardUrgencySignal__label' not in p.get('class')]
    data['date'].append(filtered_elements[0].text.split(' • ')[0])
    data['time'].append(filtered_elements[0].text.split(' • ')[1])

    data['location'].append(basic_info.get('data-event-location', '-'))
    data['category'].append(basic_info.get('data-event-category', '-'))
    # todo: solve problem with slow page loading and skeleton insteda of price
    # price = event.find_all('div', class_='discover-horizontal-event-card__price-wrapper').text


In [78]:
df = pd.DataFrame(data)

df

,title,date,time,location,category
0,THREE HOUSE HALLOWEEN.,"Tue, Oct 31",3:00 PM,"Funchal, Madeira",holiday
1,NHR Regime & Immigration Criteria Explained,Tuesday,2:00 PM,"Funchal, Madeira",business
2,DO CAMPO PARA A CIDADE,"Sat, Oct 28",9:00 PM,"Machico, Madeira",film-and-media
3,Madeira 7 Talks - 'Já Foste!',"Sun, Oct 29",2:00 PM,"Funchal, Funchal",community
4,"Lead, Thrive, Harmonize: The Ultimate Guide to...","Thu, Nov 2",9:00 AM,"Funchal, Madeira",business
5,LUIS SILVA ALMA LLANERA TOUR EN MADEIRA,"Fri, Nov 10",8:00 PM,"Funchal, Madeira",community
6,MADEIRADiG 2023,"Fri, Dec 1",8:00 PM,"Estreito da Calheta, Autonome Region Madeira",music
7,A haunting at favilla's street,"Tue, Oct 31",11:59 AM,"Funchal, Madeira",holiday
8,Community Meditation & Inspiration,Tuesday,12:30 PM,"Funchal, Madeira",spirituality
9,JONATHAN MOLY EN MADEIRA - SALA BAHÍA,"Tue, Dec 19",8:00 PM,"Funchal, Funchal",music
